# Interroger votre base de connaissances

In [ ]:
%pip install -qU wget openai

In [22]:
import json
import os
import re
import requests

import wget
from openai import OpenAI

Commencez par télécharger le document qui va vous servir de base de connaissance. Celle-ci doit être au format JSON. 

Nous prenons ici comme exemple le dataset des expériences d'usagers provenant du site de la Transformation Publique.

In [60]:
doc_url = "https://opendata.plus.transformation.gouv.fr/api/explore/v2.1/catalog/datasets/export-expa-c-riences/exports/json"
file_path = "my_database.json"

In [82]:
if not os.path.exists(file_path):
    wget.download(doc_url, out=file_path)

In [83]:
file = json.load(open(file_path))

print("Exemple de fichier :", file[0])
print("Poids du fichier :", os.path.getsize(file_path) // 1024, "MB")

Exemple de fichier : {'id_experience': 280027, 'etat_experience': 'A traiter', 'statut_de_l_experience': 'Expérience en attente de réponse', 'ecrit_le': '2019-06-20', 'date_de_publication': '2019-06-21', 'pseudonyme_usager': 'FLORIAN H.', 'titre': 'ANAH - programme habiter mieux', 'description': 'Nous avons déposé un dossier complet à l\'Anah en date du 18 décembre 2018 mais depuis cette date, il n\' y a pas encore eu de "commission" pour valider notre dossier. Fin mars, on nous avait annoncé une commission pour le 16 juin et hier, nous avons appris qu\'elle a été, une énième fois, reportée... sans avoir plus d\'infos.Pourquoi une telle situation ?', 'poste_comme': 'La personne concernée', 'ressenti_usager': 'Négatif', 'code_insee_departement_usager': None, 'intitule_departement_usager': None, 'intitule_region_usager': None, 'code_insee_region_usager': None, 'pays': 'France', 'id_typologie_1': 'vAjIgQajQN', 'intitule_typologie_1': 'ANAH', 'code_postal_typologie_1': None, 'pays_typologi

Au regard du dataset, il pose 2 problèmes : 
- sa structure ne respect par ce qui est définit dans la documentation de l'API
- son poids est supérieur à 512MB

In [84]:
base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = "YOUR_API_KEY"

client = OpenAI(base_url=base_url, api_key=api_key)

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}


Créez une collection nommée `tutorial`.

In [85]:
response = session.get(f"{base_url}/models")
response = response.json()
model = [model for model in response["data"] if model["type"] == "text-embeddings-inference"][0]["id"]

print(f"Embeddings model: {model}")

Embeddings model: BAAI/bge-m3


In [114]:
collection = "tutorial"

response = session.post(f"{base_url}/collections", json={"name": collection, "model": model})
response = response.json()
collection_id = response["id"]
print(f"Collection ID: {collection_id}")


Collection ID: 2136465a-a4a3-41f8-a7ba-019576f7f671


In [115]:
files = {"file": (os.path.basename(file_path), open(file_path, "rb"), "application/pdf")}
data = {'request': '{"collection": "%s"}' % collection_id}
response = session.post(f"{base_url}/files", data=data, files=files)
print(response.json())

{'detail': 'File size limit exceeded'}


In [116]:
formated_file = list()
for document in file[:10]:
    titre = document.get("titre", "")
    description = document.get("description", "")
    intitule = document.get("intitule_typologie_1", "")
    reponse = document.get("reponse_structure_1", "")

    text = re.sub(r"([.,;:!?])([^\s\d])", r"\1 \2", description) # Add space after punctuation
    text = re.sub(r"[\xa0\u00a0\r]", " ", text) # Remove special characters
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"\,(?!\s)", ". ", text, count=1) # Add a space after the first "," if not already followed by a space.

    formated_file.append({
        "title": titre,
        "text": f"{titre} {text} {intitule}",
        "metadata": {
            "question": text,
            "answer": reponse,
            "titre": titre,
            "intitule": intitule
        }
    })

formated_file_path = "my_database_formated.json"
json.dump(formated_file, open(formated_file_path, "w"))

In [117]:
os.path.getsize(formated_file_path) / 1024 / 1024

0.019721031188964844

In [118]:
files = {"file": (os.path.basename(formated_file_path), open(formated_file_path, "rb"), "application/pdf")}
data = {'request': '{"collection": "%s"}' % collection_id}
response = session.post(f"{base_url}/files", data=data, files=files)
print(response.json())

{'id': 'b79ed0b4-0cf0-4b70-bcdf-20d7e0843067'}


In [19]:
batch = 32
os.makedirs("my_database", exist_ok=True)
for i in range(0, len(file), batch):
    batch_file = file[i:i+batch]

    documents = []
    for document in batch_file:
        titre = document.get("titre", "")
        description = document.get("description", "")
        intitule = document.get("intitule_typologie_1", "")
        reponse = document.get("reponse_structure_1", "")

        text= re.sub(r"([.,;:!?])([^\s\d])", r"\1 \2", description) # Add space after punctuation
        text = re.sub(r"[\xa0\u00a0\r]", " ", text) # Remove special characters
        text = re.sub(r"&nbsp;", " ", text)
        text = re.sub(r"\,(?!\s)", ". ", text, count=1) # Add a space after the first "," if not already followed by a space.

        documents.append({
            "text": f"{titre} {text} {intitule}",
            "metadata": {
                "question": text,
                "answer": reponse,
                "titre": titre,
                "intitule": intitule
            }
        })

    batch_file_path = os.path.join("my_database", f"my_database_{i}.json")
    json.dump(documents, open(batch_file_path, "w"))
    assert os.path.getsize(batch_file_path) < 51200000, "Le fichier ne doit pas dépasser 512MB"

    files = {"files": (os.path.basename(batch_file_path), open(batch_file_path, "rb"), "application/pdf")}
    params = {"collection": collection_id}
    response = session.post(f"{base_url}/files", params=params, files=files)
    response.json()

In [ ]:
titre = document.get("titre", "")
description = document.get("description", "")
intitule = document.get("intitule_typologie_1", "")
reponse = document.get("reponse_structure_1", "")

# add space after punctuation
description = re.sub(r"([.,;:!?])([^\s\d])", r"\1 \2", description)

In [6]:
len(file)

80615

Puis instancier la connexion à l'API Albert.

In [2]:
file = json.load(open(file_path))

NameError: name 'json' is not defined

Vous aurez besoin pour la suite d'un modèle de langage ainsi qu'un modèle d'embeddings (pour le RAG). Pour cela vous pouvez appelez le endpoint `/v1/models` pour obtenir la liste des modèles. Les modèles de langage ont le type *text-generation* et les modèles d'embeddings le type *text-embeddings-inference*.

In [ ]:
language_model, embeddings_model = None, None

for model in client.models.list().data:
    if model.type == "text-generation" and language_model is None:
        language_model = model.id
    if model.type == "text-embeddings-inference" and embeddings_model is None:
        embeddings_model = model.id

print(f"language model: {language_model}\nembeddings model: {embeddings_model}")

Enfin pour vous importer le document dans une collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

Vous devez spécifier le modèle d'embeddings qui sera utilisé pour vectoriser votre document. Vous pouvez trouver la liste des modèles avec le endpoint `/v1/models`. Les modèles d'embeddings sont indiqués avec le type _feature-extraction_.

Le endpoint POST `/v1/files` doit retourner un status _success_.

In [ ]:
# Remove previous files
collection = "tutorial"
response = session.delete(f"{base_url}/files/{collection}")
response.status_code

In [ ]:
# Upload a file
model = "intfloat/multilingual-e5-large"
params = {"collection": collection, "embeddings_model": embeddings_model}

files = {"files": (os.path.basename(file_path), open(file_path, "rb"), "application/pdf")}
response = session.post(f"{base_url}/files", params=params, files=files)

response.json()

Vous pouvez observer les fichiers que vous avez importer dans une collection à l'aide du endpoint GET `/v1/files.`

In [ ]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")
response.json()
file_id = response.json()["data"][0]["id"]
print(file_id)

In [ ]:
prompt = "Qui est Ulrich Tan ?"
data = {"collections": [collection], "model": embeddings_model, "k": 6, "prompt": prompt}
response = session.post(url=f"{base_url}/search", json=data, headers={"Authorization": f"Bearer {api_key}"})

prompt_template = "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}\n\nDocuments :\n\n{chunks}"
chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)

Vous pouvez également faire ajouter une recherche sur internet en spécifiant "internet" dans la liste des collections.

In [ ]:
prompt = "Qui est Ulrich Tan ?"
collections = [collection, "internet"]
data = {"collections": collections, "model": embeddings_model, "k": 6, "prompt": prompt}
response = session.post(url=f"{base_url}/search", json=data, headers={"Authorization": f"Bearer {api_key}"})

prompt_template = "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}\n\nDocuments :\n\n{chunks}"
chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)